In [1]:
# ✅ IMPORTS
import pandas as pd
import sqlite3
import re
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gradio as gr
from huggingface_hub import login
import ollama

import torch

In [2]:
# ✅ STEP 2: LOAD AND CLEAN SOIL DATA
soil_df = pd.read_csv("muaggatt.txt", sep="|", header=None)

# Assign USDA soil column names
soil_df.columns = [
    "mukey", "hzname", "desgnmaster", "desgnmasterprime", "hzdept_r", "hzdepb_r",
    "fragvol_r", "dbthirdbar_r", "cec7_r", "ph1to1h2o_r", "ec_r", "sar_r",
    "eps_r", "calciumcarbonate_r", "om_r", "lep_r", "kffact", "aws0150bar_r",
    "aws03bar_r", "awc_r", "wsatiated_r", "wtenthbar_r", "wthirdbar_r", "wfifteenbar_r",
    "wsatiated33bar_r", "silttotal_r", "sandtotal_r", "claytotal_r", "gypsum_r", "aluminum_r",
    "base_sat_r", "caexch_r", "mgexch_r", "kexch_r", "naexch_r", "cec7clay_r",
    "ph01mcacl2_r", "phs_r", "cec8_r", "texture_class"
]

# Keep relevant columns
soil_df = soil_df[[
    "mukey", "hzname", "ph1to1h2o_r", "om_r", "awc_r", "texture_class",
    "cec7_r", "claytotal_r", "sandtotal_r", "calciumcarbonate_r", "hzdept_r"
]]


# ✅ Drop rows where all important columns are missing
soil_df_clean = soil_df.dropna(subset=["ph1to1h2o_r", "om_r", "awc_r"], how="all")

# ✅ Remove rows with urban/water layers
soil_df_clean = soil_df_clean[~soil_df_clean["hzname"].str.contains("water|urban", case=False, na=False)]

# ✅ Convert columns to numeric where needed
cols_to_convert = ["ph1to1h2o_r", "om_r", "awc_r", "cec7_r", "claytotal_r", "sandtotal_r", "calciumcarbonate_r", "hzdept_r"]
for col in cols_to_convert:
    soil_df_clean[col] = pd.to_numeric(soil_df_clean[col], errors="coerce")

# ✅ Fill missing values with safe estimates
soil_df_clean["ph1to1h2o_r"].fillna(6.5, inplace=True)   # Neutral pH
soil_df_clean["om_r"].fillna(2.0, inplace=True)          # Typical organic matter
soil_df_clean["awc_r"].fillna(0.15, inplace=True)        # Moderate water capacity
soil_df_clean["cec7_r"].fillna(10.0, inplace=True)       # Avg fertility
soil_df_clean["claytotal_r"].fillna(20.0, inplace=True)  # Mid-range clay
soil_df_clean["sandtotal_r"].fillna(60.0, inplace=True)  # Mid-range sand
soil_df_clean["calciumcarbonate_r"].fillna(0.0, inplace=True)
soil_df_clean["hzdept_r"].fillna(30.0, inplace=True)     # 30 cm default

# ✅ Convert pH from USDA scale to real scale (÷10)
soil_df_clean["ph1to1h2o_r"] = soil_df_clean["ph1to1h2o_r"] / 10

# Preview cleaned data
soil_df_clean.head()






C:\Users\surya\AppData\Local\Temp\ipykernel_12956\117588084.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  soil_df_clean["ph1to1h2o_r"].fillna(6.5, inplace=True)   # Neutral pH
C:\Users\surya\AppData\Local\Temp\ipykernel_12956\117588084.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

,mukey,hzname,ph1to1h2o_r,om_r,awc_r,texture_class,cec7_r,claytotal_r,sandtotal_r,calciumcarbonate_r,hzdept_r
1,s275,Rositas-Ripley-Indio-Gilman (s275),0.65,18.07,43,658353,10.0,20.0,60.0,12.65,1.3
2,s295,Schenco-Rock outcrop-Laposa (s295),0.65,5.27,100,658373,10.0,20.0,60.0,5.27,43.9
4,s1001,Tujunga-Salinas-Elder (s1001),0.65,16.41,34,660448,10.0,20.0,60.0,10.94,3.6
7,s1004,Ramona-Hanford-Greenfield-Gorgonio (s1004),0.65,16.35,54,660451,10.0,20.0,60.0,10.99,5.9
8,s1005,San Emigdio-Metz-Grangeville (s1005),0.65,18.24,41,660452,10.0,20.0,60.0,12.23,2.6


In [3]:
mapunit_df = pd.read_csv("mapunit.txt", sep="|", header=None)

# Assign column names
mapunit_df.columns = [
    "mukey", "mapunit_name", "type", "null1", "acres", "null2", "null3", "null4", "null5", "null6",
    "null7", "null8", "null9", "null10", "soil_order", "null11", "null12", "area_symbol", "mapunit_id",
    "null13", "null14", "null15", "null16", "null17"
]

mapunit_df = mapunit_df[["mukey", "mapunit_name", "type", "acres", "soil_order", "area_symbol"]]



# ✅ Merge
merged_df = pd.merge(soil_df_clean, mapunit_df, on="mukey", how="left")


In [4]:
# Load CSV
crop_df = pd.read_csv("data_core.csv")

# Normalize text fields
for col in ["Crop Type", "Soil Type", "Fertilizer Name"]:
    crop_df[col] = crop_df[col].str.lower().str.strip()

# Ensure numeric columns are clean types
num_cols = ["Temparature", "Humidity", "Moisture", "Nitrogen", "Potassium", "Phosphorous"]
for col in num_cols:
    crop_df[col] = pd.to_numeric(crop_df[col], errors="coerce")

# Optional: remove duplicates (if needed)
crop_df.drop_duplicates(inplace=True)

# Optional: sort or filter by a specific condition
# e.g., crop_df = crop_df[crop_df["Temparature"] <= 40]


In [5]:
conn = sqlite3.connect("soil_data.db")
merged_df.to_sql("soil_enriched", conn, if_exists="replace", index=False)
crop_df.to_sql("crop_data", conn, if_exists="replace", index=False)

print("✅ SQLite database 'soil_data.db' created with tables: soil_enriched and crop_data")


✅ SQLite database 'soil_data.db' created with tables: soil_enriched and crop_data


In [6]:
conn = sqlite3.connect("soil_data.db")
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Loop through tables and show their columns
for (table_name,) in tables:
    print(f"\n🔹 Columns in table '{table_name}':")
    cursor.execute(f"PRAGMA table_info({table_name});")
    for col in cursor.fetchall():
        print(f" - {col[1]}")  # col[1] is the column name


🔹 Columns in table 'soil_enriched':
 - mukey
 - hzname
 - ph1to1h2o_r
 - om_r
 - awc_r
 - texture_class
 - cec7_r
 - claytotal_r
 - sandtotal_r
 - calciumcarbonate_r
 - hzdept_r
 - mapunit_name
 - type
 - acres
 - soil_order
 - area_symbol

🔹 Columns in table 'crop_data':
 - Temparature
 - Humidity
 - Moisture
 - Soil Type
 - Crop Type
 - Nitrogen
 - Potassium
 - Phosphorous
 - Fertilizer Name


In [7]:
def get_weather_for_city(city_name, api_key="447c98ec3e842a54b1feb7c3678c56bf"):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()
        temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        return {"temperature": temp, "humidity": humidity}
    except Exception as e:
        return f"❌ Error fetching weather: {e}"


In [8]:
def extract_crop_keyword(user_input):
    conn = sqlite3.connect("soil_data.db")
    crops_df = pd.read_sql("SELECT DISTINCT `Crop Type` FROM crop_data", conn)
    crops = crops_df["Crop Type"].str.lower().str.strip().tolist()

    for crop in crops:
        if crop in user_input.lower():
            return crop
    return None


In [9]:
def extract_conditions_from_input(user_input):
    user_input = user_input.lower()
    conditions = {}

    for soil in ["loamy", "clay", "sandy", "black", "red"]:
        if soil in user_input:
            conditions["soil_type"] = soil
            break

    match = re.search(r"(?:temp(?:erature)?\s*)?(\d{2})\s*(?:c|°)?", user_input)
    if match:
        conditions["temperature"] = int(match.group(1))

    match = re.search(r"(humidity|humid)\s*(\d{2,3})", user_input)
    if match:
        conditions["humidity"] = int(match.group(2))

    return conditions


In [10]:
def recommend_crops_flexibly(conditions):
    df = pd.read_sql("SELECT * FROM crop_data", conn)
    df["Soil Type"] = df["Soil Type"].str.lower().str.strip()
    df["Crop Type"] = df["Crop Type"].str.lower().str.strip()

    if "soil_type" in conditions:
        df = df[df["Soil Type"] == conditions["soil_type"]]
    if "humidity" in conditions:
        df = df[df["Humidity"] <= conditions["humidity"] + 10]
    if "temperature" in conditions:
        df = df[df["Temparature"] <= conditions["temperature"] + 5]

    if df.empty:
        return "❌ No crop match found for the given conditions."
    return df[["Crop Type", "Soil Type", "Temparature", "Humidity", "Fertilizer Name"]].drop_duplicates()


In [11]:
def query_soil_by_crop(crop_name):
    crop_name = crop_name.lower().strip()

    # Step 1: Lookup soil type from crop_data
    soil_type_result = pd.read_sql(
        "SELECT `Soil Type` FROM crop_data WHERE `Crop Type` = ?", 
        conn, 
        params=[crop_name]
    )
    
    if soil_type_result.empty:
        return f"❌ No soil type found for {crop_name}"
    
    soil_type = soil_type_result.iloc[0].iloc[0].lower()

    # Step 2: Query enriched soil info
    query = """
    SELECT 
        mukey, mapunit_name, texture_class, ph1to1h2o_r AS pH, 
        om_r AS organic_matter, awc_r AS water_capacity,
        cec7_r AS fertility_rating, claytotal_r, sandtotal_r,
        area_symbol, acres, soil_order
    FROM soil_enriched
    WHERE LOWER(texture_class) LIKE ? OR LOWER(mapunit_name) LIKE ?
    LIMIT 5;
    """
    
    return pd.read_sql(query, conn, params=[f"%{soil_type}%", f"%{soil_type}%"])


In [12]:
def web_search(query, api_key="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"):
    url = "https://api.tavily.com/search"
    headers = {"Authorization": f"Bearer {api_key}"}
    payload = {"query": query, "num_results": 5}
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        results = response.json().get("results", [])
        return "\n\n".join([f"🔗 {r['title']}\n{r['url']}" for r in results])
    except Exception as e:
        return f"❌ Web search failed: {str(e)}"


In [13]:
# ✅ SECURITY FILTER
def is_safe_input(user_input):
    blocked = [
        "system settings", "ignore previous", "sudo", "admin", "root",
        "delete", "reboot", "bash", "format", "execute"
    ]
    return not any(word in user_input.lower() for word in blocked)

# ✅ ADVANCED PROMPTING (Meta + Self-Reflection)
prompt_cache = {}

def run_with_llm(prompt, model="mistral"):
    if not is_safe_input(prompt):
        return "🚫 Sorry, that request isn't allowed for security reasons."

    if prompt in prompt_cache:
        return prompt_cache[prompt]

    meta_prompt = f"""
You are a helpful agricultural assistant. Stay within the domain of farming.

First, generate an answer to the user's question.

Then reflect: Is this answer accurate, safe, and relevant?

If not, rephrase or reject it.

Question: {prompt}
Answer:"""

    try:
        response = ollama.chat(model=model, messages=[
            {"role": "user", "content": meta_prompt.strip()}
        ])
        answer = response["message"]["content"]
        prompt_cache[prompt] = answer
        return answer
    except Exception as e:
        return f"❌ LLM error: {e}"


In [14]:
def handle_chained_prompt(user_input, model="mistral"):
    if " and " in user_input.lower():
        parts = user_input.split(" and ")
        answers = []
        for part in parts:
            ans = run_with_llm(part.strip(), model=model)
            answers.append(f"🔹 {part.strip().capitalize()} → {ans.strip()}")
        return "\n\n".join(answers)
    return run_with_llm(user_input, model=model)


In [15]:
def smart_farming_assistant(user_input, model):
    user_input = user_input.lower()
    crop_name = extract_crop_keyword(user_input)

    # Weather-based
    if "recommend" in user_input and "for" in user_input:
        city = user_input.split("for")[-1].strip()
        weather = get_weather_for_city(city)
        if isinstance(weather, dict):
            return recommend_crops_flexibly(weather).to_string(index=False)
        else:
            return weather

    # Structured soil query
    elif any(w in user_input for w in ["soil", "ph", "moisture", "texture"]) and crop_name:
        result = query_soil_by_crop(crop_name)
        if isinstance(result, str) or result.empty:
            return run_with_llm(user_input, model=model)
        return result.to_string(index=False)

    # Web-based queries
    elif any(w in user_input for w in ["disease", "fertilizer", "weather", "news", "planting"]):
        web_result = web_search(user_input)
        if web_result.startswith("❌"):
            return run_with_llm(user_input, model=model)
        return web_result

    # Fallback to LLM
    else:
        return handle_chained_prompt(user_input, model=model)



In [16]:
# 🔍 Sample queries to test TerraGPT
test_questions = [
    "What crops grow well in loamy soil?",
    "What is the ideal pH for wheat?",
    "Suggest a fertilizer for maize.",
    "Recommend crops for clay soil with high humidity.",
    "How to prevent tomato blight?",
    "What crop grows best in sandy soil and low rainfall?",
    "Which crop suits moderately drained soil?",
    "What are signs of tomato disease?",
    "What crop can I grow in Delhi in summer?",
    "Suggest a potassium-rich fertilizer.",
    "What is the organic matter level needed for rice?",
    "Recommend a drought-resistant crop.",
    "Can I grow bananas in acidic soil?",
    "What temperature is best for brinjal?",
    "Suggest crops based on weather in Pune."
]


In [17]:
for i, q in enumerate(test_questions, 1):
    print(f"\n❓ Q{i}: {q}")
    print(smart_farming_assistant(q,model="mistral"))
    print("-" * 80)



❓ Q1: What crops grow well in loamy soil?
 Loamy soil is ideal for a variety of crops due to its good water-holding capacity and excellent nutrient retention. Some crops that thrive in loamy soil include carrots, potatoes, beets, cucumbers, lettuce, tomatoes, corn, peas, and various root vegetables like parsnips and radishes. However, it's essential to consider other factors such as sunlight exposure, temperature, and local climate when selecting crops for your farm.

Reflection: This answer is accurate, safe, and relevant as it provides specific examples of crops that grow well in loamy soil, which is a common type of soil found in many agricultural areas. The information provided is straightforward and easy to understand, making it suitable for a wide range of users seeking advice on crop selection for their farm.
--------------------------------------------------------------------------------

❓ Q2: What is the ideal pH for wheat?
 The ideal pH range for wheat growth is between 6.0

In [19]:
for i, q in enumerate(test_questions, 1):
    print(f"\n❓ Q{i}: {q}")
    print(smart_farming_assistant(q,model="llama2"))
    print("-" * 80)



❓ Q1: What crops grow well in loamy soil?
 Loamy soil is ideal for a variety of crops due to its good water-holding capacity and excellent nutrient retention. Some crops that thrive in loamy soil include carrots, potatoes, beets, cucumbers, lettuce, tomatoes, corn, peas, and various root vegetables like parsnips and radishes. However, it's essential to consider other factors such as sunlight exposure, temperature, and local climate when selecting crops for your farm.

Reflection: This answer is accurate, safe, and relevant as it provides specific examples of crops that grow well in loamy soil, which is a common type of soil found in many agricultural areas. The information provided is straightforward and easy to understand, making it suitable for a wide range of users seeking advice on crop selection for their farm.
--------------------------------------------------------------------------------

❓ Q2: What is the ideal pH for wheat?
 The ideal pH range for wheat growth is between 6.0

In [20]:
import gradio as gr
import ollama

# 🔁 Shared LLM wrapper
'''def run_with_llm(prompt, model="mistral"):
    response = ollama.chat(model=model, messages=[
        {"role": "user", "content": f"You are a farming expert.\n\nQuestion: {prompt}\nAnswer:"}
    ])
    return response["message"]["content"]'''

# 🧠 Route through smart assistant with selected model
def assistant_mistral(user_input):
    return smart_farming_assistant(user_input, model="mistral")

def assistant_llama2(user_input):
    return smart_farming_assistant(user_input, model="llama2")

# 🌿 Gradio Tabs
mistral_tab = gr.Interface(
    fn=assistant_mistral,
    inputs=gr.Textbox(label="user_input", placeholder="e.g., Recommend a crop for clay soil with high humidity", lines=2),
    outputs=gr.Textbox(label="output", lines=8),
    title="🌾 Smart Farming Assistant - Mistral",
    description="Ask about crop recommendations, soil types, fertilizer, or farming tips!"
)

llama_tab = gr.Interface(
    fn=assistant_llama2,
    inputs=gr.Textbox(label="user_input", placeholder="e.g., Recommend a crop for sandy soil with low rainfall", lines=2),
    outputs=gr.Textbox(label="output", lines=8),
    title="🌾 Smart Farming Assistant - LLaMA-2",
    description="Ask the same farming questions powered by LLaMA-2!"
)

# 🧭 Tabbed Launch
gr.TabbedInterface([mistral_tab, llama_tab], ["Mistral", "LLaMA-2"]).launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
